# Topic disaggregation

### Repeat the same process on data previously disaggregated according to the categories of the vote

In [2]:
import pandas as pd
import networkx as nx
import numpy as np
import re

import glob, os
import itertools
from tqdm import tqdm

import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import seaborn as sns
from shapely.geometry import LineString
from matplotlib.lines import Line2D

import shutil

In [3]:
import sys
sys.path.insert(0, '../local/bin')

Merge categories into "clustered topics"

In [4]:
def merge_congress_files(folder_names, merged_folder_name):
    # Create the merged folder if it doesn't exist
    if not os.path.exists(merged_folder_name):
        os.makedirs(merged_folder_name)

    for category in folder_names:
        category_folder = os.path.join(os.getcwd(), category)
        merged_folder = os.path.join(os.getcwd(), merged_folder_name)

        # Iterate through the files in the category folder
        for root, _, files in os.walk(category_folder):
            for file in files:
                if file.startswith("congress_") and file.endswith(".csv"):
                    congress_number = int(file.split("_")[1].split(".")[0])
                    congress_file = os.path.join(root, file)
                    merged_congress_file = os.path.join(merged_folder, f"congress_{congress_number}.csv")

                    # If the file already exists in the merged folder, append data
                    if os.path.exists(merged_congress_file):
                        existing_data = pd.read_csv(merged_congress_file)
                        new_data = pd.read_csv(congress_file)
                        merged_data = pd.concat([existing_data, new_data], ignore_index=True)
                        merged_data.to_csv(merged_congress_file, index=False)
                    else:
                        # If the file doesn't exist in the merged folder, copy it
                        shutil.copy(congress_file, merged_folder)

In [ ]:
# Example usage:
category_folders = ["../dataset/votes_house_clausen/Civil_Liberties/", "../dataset/votes_house_clausen/Miscellaneous_Policy/", "../dataset/votes_house_clausen/Government_Management/"]
merged_folder = "../dataset/try_merged/"
merge_congress_files(category_folders, merged_folder)

Generate meaningful sets listing the categories belonging to each set:

In [10]:
chamber = 'house'
category_set = 'peltzman'
p = '../dataset/votes_'+chamber+'_'+category_set+'/'

In [11]:
print(p)

../dataset/votes_house_peltzman/


In [12]:
internal = [p+'Budget_General_Interest/', 
			p+'Budget_Special_Interest/',
			p+'Regulation_General_Interest/', 
			p+'Regulation_Special_Interest/', 
			p+'Domestic_Social_Policy/', 
			p+'Government_Organization/',
			p+'Internal_Organization/',
			p+'D._C./']
foreign = [	p+'Defense_Policy_Budget/',
			p+'Defense_Policy_Resolution/',
			p+'Foreign_Policy_Budget/',
			p+'Foreign_Policy_Resolution/',
		   	p+'Indian_Affairs/']

internal_folder = p+'internal/'
foreign_folder = p+'foreign/'


In [13]:
if not os.path.exists(internal_folder):
	os.makedirs(internal_folder)

if not os.path.exists(foreign_folder):
	os.makedirs(foreign_folder)

merge_congress_files(internal, internal_folder)
merge_congress_files(foreign, foreign_folder)

/tmp/ipykernel_1028/1987237459.py:21: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  new_data = pd.read_csv(congress_file)
